In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
%%capture
import os

# check if this is a colab notebook and clone the repo if it is
if 'COLAB_GPU' in os.environ:
    %cd /content/
    !git clone https://github.com/Enver-group/twitch-web-analytics
    %cd twitch-web-analytics
    !gdown --id 11IfXaA66-D7vjA2R46uAilBgOPxYmKmn
else:
    os.chdir('..')

!pip install -r requirements.txt

!pip install -e .

In [3]:
#autoreload modules
%load_ext autoreload
%autoreload 2

from src.user import User

In [19]:
df = pd.read_feather("data/streamers.feather")
df.head()

,id,name,num_followers,broadcaster_type,description,lang,last_game_played_name,view_count,profile_image_url,created_at,user_follows
0,459331509,auronplay,10874205.0,partner,feliz pero no mucho,es,Minecraft,212166590,https://static-cdn.jtvnw.net/jtv_user_pictures...,2019-09-03 14:02:49,"[210708721, 431460701, 77649106, 130065491, 42..."
1,39276140,Rubius,10483570.0,partner,se contar hasta patata,es,Propnight,222921576,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-01-12 18:46:56,"[91136321, 198363811, 210708721, 133528221, 70..."
2,48878319,TheGrefg,8708686.0,partner,"Hola, me llamo David, me quedé calvo delante d...",es,Five Nights at Freddy's,197916916,https://static-cdn.jtvnw.net/jtv_user_pictures...,2013-09-12 00:45:27,"[143776262, 675347177, 33734881, 248222879, 18..."
3,83232866,ibai,8536480.0,partner,Si lees esto que sepas que te aprecio,es,VALORANT,286994651,https://static-cdn.jtvnw.net/jtv_user_pictures...,2015-02-20 16:47:56,"[35980866, 145908612, 205218019, 91136321, 524..."
4,121510236,juansguarnizo,6013625.0,partner,"Juan Sebastián Guarnizo Algarra, más conocido ...",es,Just Chatting,127964660,https://static-cdn.jtvnw.net/jtv_user_pictures...,2016-04-11 03:34:11,"[43419527, 517536651, 476005292, 109492660, 55..."


In [35]:
# converting the df into a graph
G = nx.DiGraph()
for i,user in df.iterrows():
  G.add_node(user["id"], **user.drop(["id","user_follows"]))
  if isinstance(user["user_follows"], np.ndarray):
    G.add_edges_from([(user["id"], id_followed) for id_followed in user["user_follows"]]) 

In [36]:
# checking the correctness of the graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges(), "\n")
for i, user in enumerate(G.degree()):
  if i >10:
    break
  print(G.nodes[user[0]]["name"], "is following to", user[1], "users")

Number of nodes: 75743
Number of edges: 1368929 

auronplay is following to 4089 users
josecristo_ is following to 252 users
xXxTheFocuSxXx is following to 385 users
Ampeterby7 is following to 796 users
javiDMr10 is following to 209 users
gtv_genesis is following to 362 users
Tanizen is following to 679 users
Carola is following to 966 users
CooLifeGame is following to 1276 users
Luzu is following to 1049 users
ZormanWorld is following to 425 users
